# Notebook to work with demand forecasting query
---
---

### 

# Imports

#### Standard library imports

In [1]:
import sys
sys.path.append("../")

import os

#### Third party imports

In [2]:
import mysql.connector
import pandas as pd
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from itertools import product
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#### Local application imports

In [3]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

### 

# Getting data

##### Query data from MySQL and turn it into dataframe

In [14]:
## Function parameters
db_crds = 'mysql_dp'
sql_files_path = '../pkg_dir/sql/queries/'
sql_script = 'demand_forecast_data.sql'
sql_params = sql_params_workhrs

In [15]:
## Function execution
dfx = sql_to_df(db_crds, sql_files_path, sql_script, sql_params)

/Users/rp_mbp/Documents/educacion/maestria/berkeley_mba_meng/academics/2023_spring/analysis_and_design_databases/UCB_analysis_design_databases_DP/.venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


##### Formatting some fields

In [16]:
dfx['Purchase_Date'] = pd.to_datetime(dfx['Purchase_Date'])

### 

# Exploration

In [23]:
## Monthly sales per category
dfx.groupby(
    [
        dfx['Purchase_Date'].rename('Year').dt.year,
        dfx['Purchase_Date'].rename('Month').dt.month,
        'Category'
    ]
).agg(
    count=('Prod_ID', 'sum'),
    revenue=('List_Price', 'sum'),
)

count  revenue
Year Month Category                
2022 4     dress         8      140
     5     dress         7       75
     9     coat          7      180
     10    coat          4      130
     12    coat          2       40
2023 3     dress         6       45
     4     dress         6       10

### 

# SARIMA model

### 

# Header

### 

# *Notes*

##### References
- https://towardsdatascience.com/time-series-forecasting-with-sarima-in-python-cda5b793977b

---

---